# Clusterização de municípios pelos indicadores do IBGE

Este notebook aplica o algoritmo *k-means* para agrupar os municípios por quatro variáveis socioeconômicas:

- IDHM 2010
- PIB per capita 2021
- Densidade demográfica 2010
- Taxa de escolarização (alfabetização) 2010

Esses clusters serão depois utilizados como variáveis *dummy* em modelos de regressão para controlar perfis municipais.


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

base = pd.read_csv('../data/dados_unificados_prefeitos_200k.csv')
base.columns = [c.lstrip('﻿') for c in base.columns]

features = base[['idhm_2010', 'pib_per_capita_2021',
                 'densidade_demografica_2010', 'alfabetizacao_2010']]
features = features.dropna()


In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(features)

k = 4  # número de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(X)

features = features.assign(cluster=clusters)
features.head()


In [ ]:
cluster_dummies = pd.get_dummies(features['cluster'], prefix='cluster')
base_with_clusters = base.join(cluster_dummies)

base_with_clusters.filter(like='cluster_').head()


Os *clusters* gerados são transformados em variáveis dummy para cada município. O arquivo abaixo pode ser salvo para uso em etapas posteriores do TCC.

In [ ]:
base_with_clusters.to_csv('../data/dados_com_clusters.csv', index=False)


# Análise dos clusters

Aqui avaliamos as variáveis socioeconômicas que compõem cada grupo para entender por que os clusters foram formados.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cluster_stats = features.groupby('cluster').mean().round(2)
cluster_stats

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
cols = ['idhm_2010', 'pib_per_capita_2021', 'densidade_demografica_2010', 'alfabetizacao_2010']
for ax, col in zip(axes.ravel(), cols):
    sns.boxplot(data=features, x='cluster', y=col, ax=ax, palette='Set2')
    if col in ['pib_per_capita_2021', 'densidade_demografica_2010']:
        ax.set_yscale('log')
    ax.set_title(col)
plt.tight_layout()
plt.show()

In [ ]:
# sns.pairplot(features, vars=cols, hue='cluster', corner=True, plot_kws={'alpha':0.6})

# Plot again, with colors more distinct, and use log scale for better visibility
sns.pairplot(features, vars=cols, hue='cluster', corner=True, palette='Set2', plot_kws={'alpha': 0.6})
plt.suptitle('Pairplot of Features by Cluster', y=1.02)
plt.show()

